$20^{th}$ Apr 2021

In [1]:
import pandas as pd
import os, glob
import matplotlib.pyplot as plt
import numpy as np

from datetime import datetime

In [2]:
# df = pd.read_excel('/home/douso/Documents/TrendData/Results/COVID-19-Test_result_Reporting_01_01_2021.xlsx', header=1)

In [3]:
cols_of_interest = ['SAMPLE_NUMBER', 'AGE', 'AGE_UNIT_(DAYS/MONTHS/YEARS)', 'GENDER_(M/F)', 'OCCUPATION', 'NATIONALITY', 
           'COUNTY_OF_RESIDENCE', 'SUB_COUNTY_OF_RESIDENCE', 'HAS_TRAVEL_HOSTORY(LAST_14_DAYS)_Y/N', 'TRAVEL_FROM', 
           'QUARANTINE_FACILITY/HOSPITAL/HOMESTEAD', 'SYMTOMS_SHOWN_(COUGH;FEVER;ETC)', 'DATE_OF_SAMPLE_COLLECTION', 
           'DATE_SAMPLE_RECEIVED_IN_THE_LAB', 'RESULT', 'LAB_CONFIRMATION_DATE']

In [4]:
data_dir = '/home/douso/Documents/TrendData/Results'
files = os.listdir(data_dir)
files = sorted(files)

In [5]:
def get_trend_data(df, cols):
    df_new = df.copy()
    df_new.columns = [col.replace(' ', '_').upper() for col in df.columns]
    df_cln = df_new[df_new['SAMPLE_NUMBER'].notna()]
    return df_cln[cols]

In [6]:
# keyerrors = []
# df_raw = pd.DataFrame()
# count = 0
# for file in files:
#     try:
#         df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#         df_data = get_trend_data(df_file, 'DATE_SAMPLE_RECEIVED_IN_THE_LAB')
# #         df_raw = df_raw.append(df_data)
#         count += 1
#     except: 
#         if KeyError:
#             keyerrors.append(file)
#         elif ValueError:
#             print(f'ValueError: {file}')
#     finally: pass
# print(len(keyerrors))
# sorted(keyerrors)

In [7]:
# files = ['COVID-19-Test_result_Reporting_01_07_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_01_07_2020-B.xlsx',
#  'COVID-19-Test_result_Reporting_05_09_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_05_09_2020-B.xlsx',
#  'COVID-19-Test_result_Reporting_08_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_15_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_18_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_31_07_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_31_07_2020-B.xlsx']

In [8]:
# for file in files:
#     df = pd.read_excel(f'{data_dir}/{file}', header=1)
#     try:
#         df.insert(27, 'DATE SAMPLE RECEIVED IN THE LAB', pd.NaT)
#         df.to_excel(f'/home/douso/Documents/TrendData/Results/add_col/{file}', index=False, startrow=1)
#     except:
#         raise
#         print (f'Error: {file}')
#     finally: pass

In [9]:
# keyerrors = []
# df_raw = pd.DataFrame()
# count = 0
# for file in files:
#     try:
#         df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#         df_data = get_trend_data(df_file, cols_of_interest)
#         df_raw = df_raw.append(df_data)
#         count += 1
#     except: 
#         if KeyError:
#             keyerrors.append(file)
#         elif ValueError:
#             print(f'ValueError: {file}')
#     finally: pass
# print(len(keyerrors))
# sorted(keyerrors)

In [10]:
headers = ['TESTING LAB', 'CASE ID', 'TYPE OF CASE (INITIAL/REPEAT)',
       'SAMPLE NUMBER', 'NAME', 'ID/PASSPORT NUMBER', 'AGE',
       'AGE UNIT (DAYS/MONTHS/YEARS)', 'GENDER (M/F)', 'PHONE NUMBER',
       'OCCUPATION', 'NATIONALITY', 'COUNTY OF RESIDENCE',
       'SUB COUNTY OF RESIDENCE', 'VILLAGE/ESTATE OF RESIDENCE', 'WARD',
       'COUNTY OF DIAGNOSIS', 'HAS TRAVEL HOSTORY(LAST 14 DAYS) Y/N',
       'TRAVEL FROM', 'CONTACT WITH CASE Y/N', 'CONFIRMED CASE NAME',
       'QUARANTINE FACILITY/HOSPITAL/HOMESTEAD', 'HAVE SYMPTOMS Y/N',
       'DATE OF ONSET OF SYMPTOMS', 'SYMTOMS SHOWN (COUGH;FEVER;ETC)',
       'SAMPLE TYPE (NP SWAB, OP SWAB, SERUM SPUTUM ETC)',
       'DATE OF SAMPLE COLLECTION', 'DATE SAMPLE RECEIVED IN THE LAB',
       'RESULT', 'LAB CONFIRMATION DATE']

In [11]:
# for file in keyerrors[:-1]:
#     df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#     df_file.columns = headers
#     df_file.to_excel(f'{data_dir}/add_col2/{file}', startrow=1)

The following 60 sheets, besides some June sheets (manually fixed earlier), lacked the `DATE_SAMPLE_RECEIVED_IN_THE_LAB` column, it was added for uniformity:

- 'COVID-19-Test_result_Reporting_01_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_01_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_02_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_03_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_03_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_03_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_04_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_04_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_04_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_05_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_05_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_05_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_06_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_06_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_07_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_07_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_07_09_2020.xlsx'
- 'COVID-19-Test_result_Reporting_08_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_08_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_09_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_09_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_10_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_11_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_12_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_13_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_13_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_14_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_14_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_15_08_2020-A.xlsx'
- 'COVID-19-Test_result_Reporting_15_08_2020-B.xlsx'
- 'COVID-19-Test_result_Reporting_16_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_16_10_2020 (1).xlsx'
- 'COVID-19-Test_result_Reporting_17_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_17_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_18_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_18_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_19_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_19_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_20_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_20_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_21_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_21_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_21_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_22_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_23_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_24_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_24_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_24_08_2020-A.xlsx'
- 'COVID-19-Test_result_Reporting_24_08_2020-B.xlsx'
- 'COVID-19-Test_result_Reporting_25_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_25_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_27_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_27_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_27_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_28_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_28_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_28_08_2020.xlsx'
- 'COVID-19-Test_result_Reporting_29_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_29_07_2020.xlsx'
- 'COVID-19-Test_result_Reporting_29_08_2020.xlsx'

In [12]:
# files = ['COVID-19-Test_result_Reporting_01_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_01_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_02_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_03_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_03_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_03_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_04_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_04_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_04_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_05_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_05_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_05_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_06_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_06_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_07_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_07_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_07_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_08_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_08_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_09_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_09_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_10_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_11_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_12_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_13_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_13_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_14_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_14_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_15_08_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_15_08_2020-B.xlsx',
#  'COVID-19-Test_result_Reporting_16_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_16_10_2020 (1).xlsx',
#  'COVID-19-Test_result_Reporting_17_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_17_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_18_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_18_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_19_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_19_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_20_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_20_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_21_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_21_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_21_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_22_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_23_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_24_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_24_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_24_08_2020-A.xlsx',
#  'COVID-19-Test_result_Reporting_24_08_2020-B.xlsx',
#  'COVID-19-Test_result_Reporting_25_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_25_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_27_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_27_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_27_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_28_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_28_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_28_08_2020.xlsx',
#  'COVID-19-Test_result_Reporting_29_06_2020.xlsx',
#  'COVID-19-Test_result_Reporting_29_07_2020.xlsx',
#  'COVID-19-Test_result_Reporting_29_08_2020.xlsx']

In [13]:
# for file in files:
#     df = pd.read_excel(f'{data_dir}/{file}', header=1)
#     try:
#         df.insert(27, 'DATE SAMPLE RECEIVED IN THE LAB', np.nan)
#         df.to_excel(f'{data_dir}/add_col/{file}', index=False, startrow=1)
#     except:
#         raise
#         print (f'Error: {file}')
#     finally: pass

The follwing sheets have missing values for the `RESULT` column:
- 'COVID-19-Test_result_Reporting_20_06_2020.xlsx'
- 'COVID-19-Test_result_Reporting_16_06_2020.xlsx'

Updated column names of some of the Jun files ['GENDER_(M/F)', 'LAB_CONFIRMATION_DATE', 'RESULT']

In [14]:
# df_raw = pd.DataFrame()
# count = 0
# for file in files:
#     if file.endswith('.xlsx'):
#         try:
#             df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#             df_data = get_trend_data(df_file, cols_of_interest)
#             df_raw = df_raw.append(df_data)
#             count += 1
#         except: 
#             if KeyError:
# #                 raise
#                 print(f'KeyError: {file}')
#             elif ValueError:
#                 print(f'ValueError: {file}')
#         finally: pass
#     pass
# print(f'Files processed: {count}')
# print(f'Files in the folder: {len(files)}')

### Travel History Missing - the HOSTORY misspelling

- COVID-19-Test_result_Reporting_09_09_2020.xlsx
- COVID-19-Test_result_Reporting_11_09_2020.xlsx
- COVID-19-Test_result_Reporting_12_09_2020.xlsx
- COVID-19-Test_result_Reporting_14_09_2020.xlsx
- COVID-19-Test_result_Reporting_15_09_2020.xlsx
- COVID-19-Test_result_Reporting_18_10_2020.xlsx
- Files processed: 249
- Files in the folder: 256

In [15]:
# keyerrors = []
# # df_raw = pd.DataFrame()
# count = 0
# for file in files:
#     try:
#         df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#         df_data = get_trend_data(df_file, cols_of_interest)
# #         df_raw = df_raw.append(df_data)
#         count += 1
#     except: 
#         if KeyError:
#             keyerrors.append(file)
#         elif ValueError:
#             print(f'ValueError: {file}')
#     finally: pass
# print(len(keyerrors))
# sorted(keyerrors)

In [16]:
# files = ['COVID-19-Test_result_Reporting_09_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_11_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_12_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_14_09_2020.xlsx',
#  'COVID-19-Test_result_Reporting_15_09-2020.xlsx',
#  'COVID-19-Test_result_Reporting_18_10_2020.xlsx']

In [17]:
# for file in files:
#     df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
#     (df_file.rename(columns={'HAS TRAVEL HISTORY(LAST 14 DAYS) Y/N': 'HAS TRAVEL HOSTORY(LAST 14 DAYS) Y/N'}, errors='raise').
#     to_excel(f'{data_dir}/rename_col/{file}', index=False, startrow=1))

In [18]:
df_raw = pd.DataFrame()
count = 0
for file in files:
    if file.endswith('.xlsx'):
        try:
            df_file = pd.read_excel(f'{data_dir}/{file}', header=1)
            df_data = get_trend_data(df_file, cols_of_interest)
            df_raw = df_raw.append(df_data)
            count += 1
        except: 
            if KeyError:
#                 raise
                print(f'KeyError: {file}')
            elif ValueError:
                print(f'ValueError: {file}')
        finally: pass
    pass
print(f'Files processed: {count}')
print(f'Files in the folder: {len(files)}')

/home/douso/.local/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Files processed: 254
Files in the folder: 255


In [19]:
df_raw.head()

,SAMPLE_NUMBER,AGE,AGE_UNIT_(DAYS/MONTHS/YEARS),GENDER_(M/F),OCCUPATION,NATIONALITY,COUNTY_OF_RESIDENCE,SUB_COUNTY_OF_RESIDENCE,HAS_TRAVEL_HOSTORY(LAST_14_DAYS)_Y/N,TRAVEL_FROM,QUARANTINE_FACILITY/HOSPITAL/HOMESTEAD,SYMTOMS_SHOWN_(COUGH;FEVER;ETC),DATE_OF_SAMPLE_COLLECTION,DATE_SAMPLE_RECEIVED_IN_THE_LAB,RESULT,LAB_CONFIRMATION_DATE
0,COVC22771,58.0,Years,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,Years,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,Years,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,Years,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,Years,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [20]:
os.getenv('PWD')

'/home/douso'

### **Clean Headers**

In [21]:
cols_rename = ['S_NUM', 'AGE', 'AGE_UNIT', 'GEND', 'OCCU', 'NAT', 'COUNT_RES', 'S_COUNT_RES', 'TRAV_HIST', 
                    'TRAV_FROM', 'QUAR_PLACE', 'SYMPS', 'DT_SAM_COLL', 'DT_SAM_RECEP', 'RESULT', 'DT_CONF']

In [22]:
# df_raw.columns = cols_rename

### **Clean Sample Numbers**

In [23]:
def cov_names(x):
    x = x.upper().replace(' ', '')
    if len(x) == 8:
        x = x.replace('COVC', 'COVC0')
    elif len(x) == 7:
        x = x.replace('COVC', 'COVC00')
    elif len(x) == 6:
        x = x.replace('COVC', 'COVC000')
    elif len(x) == 5:
        x = x.replace('COVC', 'COVC0000')
    return x

In [24]:
df_raw.head()

,SAMPLE_NUMBER,AGE,AGE_UNIT_(DAYS/MONTHS/YEARS),GENDER_(M/F),OCCUPATION,NATIONALITY,COUNTY_OF_RESIDENCE,SUB_COUNTY_OF_RESIDENCE,HAS_TRAVEL_HOSTORY(LAST_14_DAYS)_Y/N,TRAVEL_FROM,QUARANTINE_FACILITY/HOSPITAL/HOMESTEAD,SYMTOMS_SHOWN_(COUGH;FEVER;ETC),DATE_OF_SAMPLE_COLLECTION,DATE_SAMPLE_RECEIVED_IN_THE_LAB,RESULT,LAB_CONFIRMATION_DATE
0,COVC22771,58.0,Years,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,Years,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,Years,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,Years,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,Years,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [25]:
df_raw.to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged.xlsx', 
              index=False, header=cols_rename)

In [26]:
# df_raw.head()

In [27]:
# df_mrg_mtdata = pd.read_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged.xlsx')

In [28]:
## df_mrg_mtdata[df_mrg_mtdata['AGE_UNIT'] != 'Years'].AGE_UNIT.unique()

In [29]:
## df_mrg_mtdata[df_mrg_mtdata['AGE_UNIT'] == 'Months']#.AGE_UNIT.unique()

In [30]:
## df_mrg_mtdata[df_mrg_mtdata['AGE_UNIT'] == 'Days']#.AGE_UNIT.unique()

In [31]:
df_mrg_mtdata = pd.read_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged.xlsx')

### **Clean Sample Name**

In [32]:
df_name = df_mrg_mtdata.assign(S_NUM=df_mrg_mtdata['S_NUM'].apply(lambda x: cov_names(x)))

In [33]:
df_name[df_name['S_NUM'].str.len() < 8]

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9011,COVE20,39,Years,M,SECURITY GUARD,Kenya,Machakos,ATHI RIVER,No,NaN,NaN,NaN,2020-09-10 00:00:00,2020-09-10 00:00:00,Negative,2020-09-11 00:00:00
9012,COVE21,50,Years,M,SECURITY GUARD,Kenya,Machakos,ATHI RIVER,No,NaN,NaN,NaN,2020-09-10 00:00:00,2020-09-10 00:00:00,Negative,2020-09-11 00:00:00
16345,NIL,40,Years,M,Nil,Kenya,Laikipia,Laikipia East,No,NaN,NTRH,NaN,2020-06-18 00:00:00,NaN,Negative,2020-06-20 00:00:00


In [34]:
df_name[df_name['S_NUM'].str.contains('COVC') == False]

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9011,COVE20,39,Years,M,SECURITY GUARD,Kenya,Machakos,ATHI RIVER,No,NaN,NaN,NaN,2020-09-10 00:00:00,2020-09-10 00:00:00,Negative,2020-09-11 00:00:00
9012,COVE21,50,Years,M,SECURITY GUARD,Kenya,Machakos,ATHI RIVER,No,NaN,NaN,NaN,2020-09-10 00:00:00,2020-09-10 00:00:00,Negative,2020-09-11 00:00:00
16345,NIL,40,Years,M,Nil,Kenya,Laikipia,Laikipia East,No,NaN,NTRH,NaN,2020-06-18 00:00:00,NaN,Negative,2020-06-20 00:00:00
23661,KEM-21-03-93738,42,Years,F,NaN,Kenya,Kiambu,Kahoho,No,NaN,NaN,NaN,2021-03-03 00:00:00,NaN,Positive,2021-05-03 00:00:00
23662,KEM-21-03-93999,55,Years,F,NaN,Kenya,Kiambu,Githunguri,No,NaN,NaN,General weakness,02/03/2021,NaN,Positive,10/03/2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23808,KEM-21-03-94425,17,Years,M,NaN,Kenya,Kiambu,Thika,No,NaN,NaN,"Cough, Runny nose",04/03/2021,NaN,Positive,11/03/2021
23809,KEM-21-03-94736,16,Years,M,NaN,Kenya,Nairobi,Roysambu,No,NaN,NaN,NaN,12/03/2021,NaN,Positive,16/03/2021
23810,KEM-21-03-94766,22,Years,M,NaN,Kenya,Kiambu,Gatundu North,No,NaN,NaN,"Headache,cough",2021-10-03 00:00:00,NaN,Positive,15/03/2021
23811,KEM-21-03-95380,60,Years,F,NaN,Kenya,Kiambu,Kabete,No,NaN,NaN,NaN,18/03/2021,NaN,Positive,19/03/2021


### **Clean Age**

In [35]:
df_name[df_name['AGE_UNIT'] == 'Days']

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [36]:
df_months = df_name[df_name['AGE_UNIT'] == 'Months']

In [37]:
df_months.head()

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
569,COVC09555,11,Months,F,NaN,Kenya,Narok,RONGAI,No,NaN,NaN,NaN,2020-09-01 00:00:00,NaN,Negative,2020-09-02 00:00:00
1150,COVC02581,1,Months,M,Inmate,Uganda,Kiambu,Thika,N,NIL,Prison,NIL,2020-07-01 00:00:00,NaN,Negative,2020-07-03 00:00:00
1578,COVC17631,10,Months,F,NaN,Kenya,Kiambu,KIKUYU,No,NaN,NaN,NaN,2020-10-27 00:00:00,2020-10-03 00:00:00,Negative,2020-11-03 00:00:00
2429,COVC18026,6,Months,M,NaN,Kenya,Kiambu,THIKA,No,NaN,NaN,CO,2020-11-03 00:00:00,2020-11-04 00:00:00,Positive,2020-11-04 00:00:00
2586,COVC18183,19,Months,M,Child,Kenya,Kiambu,LIMURU,No,NaN,NaN,NaN,2020-11-03 00:00:00,2020-11-04 00:00:00,Negative,2020-11-04 00:00:00


In [38]:
df_mn2yrs = df_months.assign(AGE=df_months['AGE'].map(lambda x: round(x / 12, 1))).replace('Months', 'Years')

In [39]:
df_mn2yrs.head()

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
569,COVC09555,0.9,Years,F,NaN,Kenya,Narok,RONGAI,No,NaN,NaN,NaN,2020-09-01 00:00:00,NaN,Negative,2020-09-02
1150,COVC02581,0.1,Years,M,Inmate,Uganda,Kiambu,Thika,N,NIL,Prison,NIL,2020-07-01 00:00:00,NaN,Negative,2020-07-03
1578,COVC17631,0.8,Years,F,NaN,Kenya,Kiambu,KIKUYU,No,NaN,NaN,NaN,2020-10-27 00:00:00,2020-10-03 00:00:00,Negative,2020-11-03
2429,COVC18026,0.5,Years,M,NaN,Kenya,Kiambu,THIKA,No,NaN,NaN,CO,2020-11-03 00:00:00,2020-11-04 00:00:00,Positive,2020-11-04
2586,COVC18183,1.6,Years,M,Child,Kenya,Kiambu,LIMURU,No,NaN,NaN,NaN,2020-11-03 00:00:00,2020-11-04 00:00:00,Negative,2020-11-04


In [40]:
df_name[df_name['AGE_UNIT'] == 'Days']#.AGE_UNIT.unique()

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [41]:
df_not_months = df_name[df_name['AGE_UNIT'] != 'Months']

In [42]:
df_not_months.shape

(23747, 16)

In [43]:
df_mn2yrs.shape

(66, 16)

In [44]:
df_years = df_not_months.append(df_mn2yrs)

In [45]:
df_years.shape

(23813, 16)

In [46]:
df_years['AGE_UNIT'].unique()

array(['Years', nan, 'years', 'Nil'], dtype=object)

In [47]:
df_years[df_years['AGE_UNIT'] == 'NO PATIENT DATASHEET']

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [48]:
df_years[df_years['AGE_UNIT'].str.contains('M', 'F') == True]

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF


In [49]:
df_years[df_years['AGE_UNIT'] == 'Nil']

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
14211,COVC04083,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14212,COVC04084,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14213,COVC04085,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Positive,2020-07-17 00:00:00
14214,COVC04086,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14215,COVC04087,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14216,COVC04088,Nil,Nil,F,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14217,COVC04089,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14218,COVC04090,Nil,Nil,F,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14219,COVC04091,Nil,Nil,M,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00
14220,COVC04092,Nil,Nil,F,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,Nil,NaN,Negative,2020-07-17 00:00:00


In [50]:
df_years_drop_age_unit = df_years.drop('AGE_UNIT', axis=1)

In [51]:
df_years_rencol = df_years_drop_age_unit.rename(columns={'AGE': 'AGE_YRS'})

In [52]:
df_years_rencol.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [53]:
df_years_rencol['AGE_YRS'].unique()

array([58, 63, 53, 52, 41, 9, 6, 39, 23, 36, 33, 42, 34, 32, 57, 56, 27,
       20, nan, 44, 26, 31, 35, 28, 37, 54, 29, 43, 24, 38, 47, 49, 40,
       51, 50, 25, 30, 45, 46, 22, 21, 48, 55, 65, 64, 60, 18, 16, 10, 17,
       15, 8, 13, 5, 12, 84, 59, 62, 68, 312, 61, 19, 2, 86, 80, 69, 72,
       3, 4, 7, 74, 11, 67, 87, 3.6, 1, 14, 1.5, 70, 2.42,
       'None indicated', 73, 6.5, 2.5, 75, 97, 'Nil', 71, 89, 96, 82, 77,
       66, 'Not indicated', 743, 78, 76, 'None', 9.5, 'nil', 79, 81,
       'none indicated', 90, 85, 91, 100, 101, 83, 92, 'not_indicated',
       5.5, 98, 'Not Indicated', '37', '36', '40', '38', '29', 94, 105,
       0.5, 'not Indicated', '30', '44', '33', '27', '26', '51', '28',
       'None Indicated', 542, 0.8, 0.9, 0.1, 1.6, 3.7, 0.2, 0.3, 2.8, 0.7,
       1.2, 2.1, 0.4, 4.5, 1.8, 2.9, 1.7, 0.6], dtype=object)

In [54]:
[x for x in df_years_rencol['AGE_YRS'].unique() if isinstance(x, str)]

['None indicated',
 'Nil',
 'Not indicated',
 'None',
 'nil',
 'none indicated',
 'not_indicated',
 'Not Indicated',
 '37',
 '36',
 '40',
 '38',
 '29',
 'not Indicated',
 '30',
 '44',
 '33',
 '27',
 '26',
 '51',
 '28',
 'None Indicated']

In [55]:
to_rep = ['None indicated',
 'Nil',
 'Not indicated',
 'None',
 'nil',
 'none indicated',
 'not_indicated',
 'Not Indicated',
 'not Indicated',
 'None Indicated',
 'NO PATIENT DATASHEET'
]

In [56]:
df_years_repnan2float = df_years_rencol.assign(AGE_YRS=df_years_rencol['AGE_YRS'].replace(to_rep, np.nan).astype(float))

In [57]:
df_years_repnan2float

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22221,COVC16833,0.8,M,NaN,Kenya,Nairobi,DONHOLM,No,NaN,NaN,NaN,2020-10-29 00:00:00,2020-10-29 00:00:00,Negative,2020-10-30 00:00:00
22477,COVC17089,0.6,M,NaN,Kenya,Mombasa,LIKONI,No,NaN,NaN,CO,2020-10-22 00:00:00,2020-10-27 00:00:00,Negative,2020-10-30 00:00:00
22999,COVC23794,0.2,M,NaN,Kenya,Kiambu,GITHUNGURI,No,NaN,NaN,H,2021-05-02 00:00:00,2021-05-05 00:00:00,Negative,2021-05-06 00:00:00
23120,COVC23131,2.5,F,NaN,Kenya,Mombasa,MVITA,No,NaN,NaN,NaN,2021-02-08 00:00:00,2021-02-11 00:00:00,Negative,2021-02-11 00:00:00


### **Clean Gender**

In [58]:
df_years_repnan2float['GEND'].unique()

array(['M', 'F', 'f', 'Female', 'Male', 'F`', 'MALE ', 'MALE', nan, 'Nil',
       'T'], dtype=object)

In [59]:
df_years_repnan2float[df_years_repnan2float['GEND'].isna()].sort_values('S_NUM')

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9390,COVC00749,42.0,NaN,Nil,Kenyan,Laikipia,Laikipia East,N,NaN,Nanyuki Teaching and Referral Hospital,Asymtomatic,9th June 2020,"June 11, 2020",Neg,2020-06-12 00:00:00
12355,COVC01037,30.0,NaN,Nil,Kenyan,Nairobi,Embaksi,N,NaN,Nil,Asymtomatic,Nil,2020-06-15 00:00:00,Neg,2020-06-15 00:00:00
17411,COVC04494,NaN,NaN,NaN,Kenya,Nyeri,NaN,No,NaN,NaN,NaN,2020-07-14 00:00:00,NaN,Negative,2020-07-23 00:00:00
17412,COVC04495,NaN,NaN,NaN,Kenya,Nyeri,NaN,No,NaN,NaN,NaN,2020-07-14 00:00:00,NaN,Negative,2020-07-23 00:00:00
17413,COVC04496,NaN,NaN,NaN,Kenya,Nyeri,NaN,No,NaN,NaN,NaN,2020-07-14 00:00:00,NaN,Negative,2020-07-23 00:00:00
22764,COVC05236,NaN,NaN,NaN,NaN,Nyeri,NaN,NaN,NaN,NaN,NaN,2020-07-28 00:00:00,NaN,Negative,2020-07-31 00:00:00
22611,COVC05255,NaN,NaN,NaN,NaN,NaN,not_indicated,No,NaN,NaN,NaN,NaN,NaN,Negative,2020-07-31 00:00:00
12294,COVC13844,NaN,NaN,NaN,NaN,Nairobi,NaN,No,NaN,NaN,NaN,2020-10-13 00:00:00,2020-10-13 00:00:00,Negative,2020-10-14 00:00:00
12295,COVC13845,NaN,NaN,NaN,NaN,Nairobi,NaN,No,NaN,NaN,NaN,2020-10-13 00:00:00,2020-10-13 00:00:00,Negative,2020-10-14 00:00:00
12296,COVC13846,NaN,NaN,NaN,NaN,Nairobi,NaN,No,NaN,NaN,NaN,2020-10-13 00:00:00,2020-10-13 00:00:00,Negative,2020-10-14 00:00:00


In [60]:
df_years_repnan2float[df_years_repnan2float['GEND'] == 'Nil']

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
10718,COVC00749,42.0,Nil,Nil,Kenyan,Laikipia,Laikipia East,N,NaN,Nanyuki Teaching and Referral Hospital,Asymtomatic,9th June 2020,"June 11, 2020",Neg,2020-06-12 00:00:00
15504,COVC01628,NaN,Nil,Nil,Nil,Nyeri,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN


COVEs:
- COVC13844
- COVC13845
- COVC13846
- COVC13847
- COVC13848
- COVC13849
- COVC13850
- COVC13851
- COVC13852
- COVC13853
- COVC16256
- COVC16257

In [61]:
df_years_repgender = df_years_repnan2float.assign(GEND=df_years_repnan2float['GEND'].replace(['f', 'Female', 'F`', 'T'], 'F')
                                                 .replace(['Male', 'MALE ', 'MALE'], 'M'))

In [62]:
df_years_repgender['GEND'].unique()

array(['M', 'F', nan, 'Nil'], dtype=object)

In [63]:
df_years_repgender['GEND'].replace('Nil', None, inplace=True)

In [64]:
df_years_repgender['GEND'].unique()

array(['M', 'F', nan], dtype=object)

In [65]:
df_years_repgender['OCCU'].unique()

array(['HCW', 'CEC', 'Psychologist', ..., 'Accountant CCN', 'prisoner',
       'Child accompanying inmate mothe'], dtype=object)

### **Clean Nationality**

In [66]:
df_years_repgender['NAT'].unique()

array(['Kenya', 'Ethiopia', 'Somalia', 'Uganda', 'South Africa', 'India',
       'United States of America (USA)', 'Germany', 'Sudan',
       'South Sudan', nan, 'Lebanon', 'Sweden', 'Benin',
       'United Kingdom (UK)', 'Netherlands', 'Cuba', 'Kenyan', 'Indian',
       'Ghana', 'Zimbabwe', 'Rwanda', 'Mauritania', 'Denmark', 'Pakistan',
       'Tanzania', 'Canada', 'Congo, Democratic Republic of the',
       'Burundi', 'Italy', 'Australia', 'Colombia', 'Hungary', 'Djibouti',
       'Finland', 'kenya', 'Yemen', 'Turkey', 'Philippines', 'Nigeria',
       'Japan', 'Ireland', 'Tunisia', 'Burkina Faso', 'Brazil',
       'Portugal', 'Cameroon', 'Bangladesh', 'Madagascar',
       "Cote d'Ivoire", 'Egypt', 'China', 'KENYAN', 'Bulgaria', 'Nil',
       'Eritrea', 'Austria', 'Malawi', 'Congo, Republic of the',
       'Sri Lanka', 'United States', 'Jordan'], dtype=object)

In [67]:
df_years_repnat = df_years_repgender.assign(NAT=df_years_repgender['NAT'].replace('Nil', None)
                                            .replace(['United States', 'Indian'
                                                      , 'Congo, Democratic Republic of the'
                                                     , 'Congo, Republic of the'] 
                                                     , ['United States of America (USA)','India'
                                                        , 'Democratic Republic of the Congo',
                                                       'Republic of the Congo'])
                                            .map(lambda x: str(x).replace(x, 'Kenya') if ('enya' in str(x).lower()) else x))

In [68]:
df_years_repnat['NAT'].unique()

array(['Kenya', 'Ethiopia', 'Somalia', 'Uganda', 'South Africa', 'India',
       'United States of America (USA)', 'Germany', 'Sudan',
       'South Sudan', nan, 'Lebanon', 'Sweden', 'Benin',
       'United Kingdom (UK)', 'Netherlands', 'Cuba', 'Ghana', 'Zimbabwe',
       'Rwanda', 'Mauritania', 'Denmark', 'Pakistan', 'Tanzania',
       'Canada', 'Democratic Republic of the Congo', 'Burundi', 'Italy',
       'Australia', 'Colombia', 'Hungary', 'Djibouti', 'Finland', 'Yemen',
       'Turkey', 'Philippines', 'Nigeria', 'Japan', 'Ireland', 'Tunisia',
       'Burkina Faso', 'Brazil', 'Portugal', 'Cameroon', 'Bangladesh',
       'Madagascar', "Cote d'Ivoire", 'Egypt', 'China', 'Bulgaria',
       'Eritrea', 'Austria', 'Malawi', 'Republic of the Congo',
       'Sri Lanka', 'Jordan'], dtype=object)

### **Clean County of Res**

In [69]:
df_years_repnat.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,LAVINGTON,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,THIKA,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,THIKA,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,F,Business,Kenya,Kiambu,THIKA,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,GATUNDU NORTH,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [70]:
len(df_years_repnat['S_COUNT_RES'].str.lower().unique())

875

In [71]:
len((df_years_repnat['S_COUNT_RES']
     .replace(['Nil', 'nan', '', 'Nan', 'Muranga ', 'Murang`a ']
                            , [np.nan, np.nan, np.nan, np.nan, "Murang'a", "Murang'a"])
                   .apply(lambda x: str(x).strip().capitalize())
     .unique()))

798

In [72]:
s_counties = (df_years_repnat['S_COUNT_RES']
     .replace(['Nil', 'nan', '', 'Nan', 'Muranga ', 'Murang`a ']
                            , [np.nan, np.nan, np.nan, np.nan, "Murang'a", "Murang'a"])
                   .apply(lambda x: str(x).strip().capitalize()))

In [73]:
# sorted([s for s in s_counties.unique()])

In [74]:
counties = (df_years_repnat['COUNT_RES']
                   .replace(['Nil', 'nan', '', 'Nan', 'Muranga ', 'Murang`a ']
                            , [np.nan, np.nan, np.nan, np.nan, "Murang'a", "Murang'a"])
                   .apply(lambda x: str(x).strip().capitalize()))

In [75]:
counties.unique()

array(['Nairobi', 'Kiambu', 'Kajiado', 'Machakos', 'Laikipia', 'Marsabit',
       'Makueni', "Murang'a", 'Isiolo', 'Garissa', 'Nakuru', 'Samburu',
       'Mombasa', 'Nan', 'Nyeri', 'Kisii', 'Tana river', 'Kitui', 'Wajir',
       'Nyandarua', 'Kirinyaga', 'Meru', 'Bungoma', 'Kakamega', 'Nandi',
       'Siaya', 'Kilifi', 'Kwale', 'Homabay', 'Muranga', 'Uganda',
       'Mandera', 'Uasin gishu', 'Kampala', 'Niarobi', 'Kisumu',
       'Badiere', 'Embu', 'Busia', 'Kigali', 'Mukono/uganda', 'Congo',
       'Nil', 'Vihiga', 'Narok', 'Kericho', 'Trans nzoia',
       'Elgeyo marakwet', 'Kajido', 'Taita taveta', 'Migori', 'Bomet'],
      dtype=object)

In [76]:
df_years_county = (df_years_repnat.assign(COUNT_RES=counties)).assign(S_COUNT_RES=s_counties)

In [77]:
df_years_county['COUNT_RES'].unique()

array(['Nairobi', 'Kiambu', 'Kajiado', 'Machakos', 'Laikipia', 'Marsabit',
       'Makueni', "Murang'a", 'Isiolo', 'Garissa', 'Nakuru', 'Samburu',
       'Mombasa', 'Nan', 'Nyeri', 'Kisii', 'Tana river', 'Kitui', 'Wajir',
       'Nyandarua', 'Kirinyaga', 'Meru', 'Bungoma', 'Kakamega', 'Nandi',
       'Siaya', 'Kilifi', 'Kwale', 'Homabay', 'Muranga', 'Uganda',
       'Mandera', 'Uasin gishu', 'Kampala', 'Niarobi', 'Kisumu',
       'Badiere', 'Embu', 'Busia', 'Kigali', 'Mukono/uganda', 'Congo',
       'Nil', 'Vihiga', 'Narok', 'Kericho', 'Trans nzoia',
       'Elgeyo marakwet', 'Kajido', 'Taita taveta', 'Migori', 'Bomet'],
      dtype=object)

### **Clean Travel History**

In [78]:
df_years_county.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,Lavington,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,Thika,No,NaN,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,Thika,No,NaN,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,F,Business,Kenya,Kiambu,Thika,No,NaN,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,Gatundu north,No,NaN,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


In [79]:
df_years_county['TRAV_HIST'].unique()

array(['No', nan, 'Yes', 'N', 'JH', 'no', 'NO ', 'Nil', 'YES', 'NONE ',
       'NONE', 'Y', 'y', 'yes'], dtype=object)

In [80]:
df_travl = df_years_county.assign(TRAV_HIST=df_years_county['TRAV_HIST'].replace([np.nan, 'N', 'no', 'NO', 'Nil', 'NONE ', 'NONE'], 'No')
           .replace(['YES', 'Y', 'y', 'yes'], 'Yes'))


In [81]:
df_travl['TRAV_FROM'].unique()

array([nan, 'ELDORET', 'NAIVASHA', 'NAIROBI;NYERI;NANYUKI', 'NIL',
       'Ruaka', 'Ngoingwa', 'Limuru', 'Thika', 'Embakasi', 'Uganda',
       'Dubai', 'Nil', 'Not indicated on form ', 'TORORO UGANDA',
       'UGANDA', 'AWASI', 'KIGALI', 'ENTEBE', 'NAIROBI', 'JINJA',
       'KISUMU', 'KAMPALA', 'KASESE', 'KILIFI COUNTY', 'KWALE COUNTY',
       'TORORO', 'KAMPALA UGANDA', 'RWANDA', 'JINJA UGANDA', 'KAJIADO',
       'TANZANIA', 'NOT APPLICABLE ', 'SOUTH SUDAN ', 'NYERI',
       'KWALE;KILIFI', 'KAKAMEGA', 'KWALE', 'Mombasa', 'Nairobi',
       'Eldoret', 'Malindi', 'Uasin Gishu', 'N', 'Machakos', 'No',
       'KILIFI/NAIROBI', 'NANYUKI SAGANA', 'THARAKA NITHI',
       'SHIMBA HILLS', 'KILIFI', 'MACHAKOS', 'Netherlands', 'TAVETA',
       'Mombasa (Stayed in Mtwapa for 4 days)', 'KINANGO MSAMBWENI',
       'MERU', 'THIKA', 'MUHONIA', 'KASESE UGANDA', 'KIRINYAGA', 'KIAMBU',
       'DAR ES SAALAM', 'TAITA TAVETA', 'Tanzania', 'not_indicated',
       'ETHIOPIA', 'Nyeri town', 'MALABA', 'NAKURU

In [82]:
df_travl2 = (df_travl.assign(TRAV_FROM=df_travl['TRAV_FROM'].replace(['NIL', 'Nil', 'Not indicated on form '
                  , 'NOT APPLICABLE ', 'N', 'N', 'No', 'not_indicated', 'Not applicable', 'Not indicated'],np.nan)
                    .replace(['Mombasa (stayed in mtwapa for 4 days)', 'KINANGO MSAMBWENI', 'Nan'], 
                    ['Mtwapa', 'Kinangop;Msambweni', np.nan])
                    .apply(lambda x: str(x).replace('/', ';')
                    .replace('and', ';')
                    .replace('-', ';')
                    .replace(',', ';')
                    .replace(' ; ', ';')
                    .replace('; ', ';')
                    .strip().capitalize())))

In [83]:
df_travl2['TRAV_FROM'].unique()



array(['Nan', 'Eldoret', 'Naivasha', 'Nairobi;nyeri;nanyuki', 'Ruaka',
       'Ngoingwa', 'Limuru', 'Thika', 'Embakasi', 'Ug;a', 'Dubai',
       'Tororo uganda', 'Uganda', 'Awasi', 'Kigali', 'Entebe', 'Nairobi',
       'Jinja', 'Kisumu', 'Kampala', 'Kasese', 'Kilifi county',
       'Kwale county', 'Tororo', 'Kampala uganda', 'Rwanda',
       'Jinja uganda', 'Kajiado', 'Tanzania', 'South sudan', 'Nyeri',
       'Kwale;kilifi', 'Kakamega', 'Kwale', 'Mombasa', 'Malindi',
       'Uasin gishu', 'Machakos', 'Kilifi;nairobi', 'Nanyuki sagana',
       'Tharaka nithi', 'Shimba hills', 'Kilifi', 'Netherl;s', 'Taveta',
       'Mombasa (stayed in mtwapa for 4 days)', 'Kinangop;msambweni',
       'Meru', 'Muhonia', 'Kasese uganda', 'Kirinyaga', 'Kiambu',
       'Dar es saalam', 'Taita taveta', 'Ethiopia', 'Nyeri town',
       'Malaba', 'Nakuru', 'Narok', 'Within kenya', 'Tharakanithi',
       'Nyahururu', 'Nairobi;kisumu', 'Nairobi;nyahururu', 'Nakuru;meru',
       'Busia', 'Bungoma', 'Nanyuki', 'K

In [84]:
df_travl2.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,Lavington,No,Nan,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,Thika,No,Nan,NaN,FLU,2020-12-31 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,Thika,No,Nan,NaN,NaN,2021-01-01 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
3,COVC22774,52.0,F,Business,Kenya,Kiambu,Thika,No,Nan,NaN,CO;DIB,2021-01-02 00:00:00,2021-01-01 00:00:00,Negative,2021-01-01 00:00:00
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,Gatundu north,No,Nan,NaN,CO;DIB,2021-01-03 00:00:00,2021-01-01 00:00:00,Positive,2021-01-01 00:00:00


### **Clean Dates**

In [85]:
def my_date_parser(x):
    try:
        if type(x) == pd.Timestamp:
            x = x.strftime('%d-%m-%Y')
        elif type(x) != pd.Timestamp and type(x) != (int, float, pd.NaT):
            x = pd.to_datetime(x, errors='ignore', dayfirst=True, infer_datetime_format=True).strftime('%d-%m-%Y')
        elif x in ['None indicated', 'NIL', 'Not indicated on form ',
                    'Nil', 'None indicated', 'Leaked Sample - Empty', 'nan']:
            x = pd.to_datetime(x, errors='coerce', dayfirst=True)
    except (ValueError, AttributeError):
        x = x
    return x

In [86]:
df_dt1 = df_travl2.assign(DT_SAM_COLL=df_travl2['DT_SAM_COLL'].apply(lambda x: my_date_parser(x)))

In [87]:
df_dt2 = df_dt1.assign(DT_SAM_RECEP=df_dt1['DT_SAM_RECEP'].apply(lambda x: my_date_parser(x)))

In [88]:
df_dts_cln = df_dt2.assign(DT_CONF=df_dt2['DT_CONF'].apply(lambda x: my_date_parser(x)))

In [89]:
df_dts_cln.head()

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
0,COVC22771,58.0,M,HCW,Kenya,Nairobi,Lavington,No,Nan,NaN,NaN,01-01-2021,01-01-2021,Negative,01-01-2021
1,COVC22772,63.0,M,CEC,Kenya,Kiambu,Thika,No,Nan,NaN,FLU,31-12-2020,01-01-2021,Negative,01-01-2021
2,COVC22773,53.0,F,Psychologist,Kenya,Kiambu,Thika,No,Nan,NaN,NaN,01-01-2021,01-01-2021,Negative,01-01-2021
3,COVC22774,52.0,F,Business,Kenya,Kiambu,Thika,No,Nan,NaN,CO;DIB,02-01-2021,01-01-2021,Negative,01-01-2021
4,COVC22775,41.0,F,HCW,Kenya,Kiambu,Gatundu north,No,Nan,NaN,CO;DIB,03-01-2021,01-01-2021,Positive,01-01-2021


### **Clean Results**

In [90]:
df_dts_cln['RESULT'].unique()

array(['Negative', 'Positive', nan, 'neg', 'pos', 'negative', 'positive',
       'Neg', 'Pos', 'Inconclusive', 'MS', 'Positve', 'Positive '],
      dtype=object)

In [91]:
df_dts_cln['RESULT'].map(lambda x: x.strip().lower() if (isinstance(x, str)) else x).unique()

array(['negative', 'positive', nan, 'neg', 'pos', 'inconclusive', 'ms',
       'positve'], dtype=object)

In [92]:
def cln_results(x):
    if isinstance(x, str):
        x = x.strip().lower()
        if 'neg' in x:
            x = x.replace(x, 'Negative')
        elif 'pos' in x:
            x = x.replace(x, 'Positive')
        elif x == 'inconclusive':
            x = x.replace(x, 'Inconclusive')
    return x
        

In [93]:
df_dts_cln['RESULT'] = df_dts_cln['RESULT'].map(lambda x: cln_results(x))

In [94]:
df_dts_cln['RESULT'].unique()

array(['Negative', 'Positive', nan, 'Inconclusive', 'ms'], dtype=object)

In [95]:
df_dts_cln.to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged-cln.xlsx', index=False)

In [96]:
df_dts_cln[df_dts_cln['RESULT'] == 'Positive'].to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged-cln-pos.xlsx', index=False)

In [97]:
df_dts_cln_pos = df_dts_cln[df_dts_cln['RESULT'] == 'Positive']

In [98]:
df_dts_cln[df_dts_cln['S_NUM'].str.contains('COVC') == False]

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9011,COVE20,39.0,M,SECURITY GUARD,Kenya,Machakos,Athi river,No,Nan,NaN,NaN,10-09-2020,10-09-2020,Negative,11-09-2020
9012,COVE21,50.0,M,SECURITY GUARD,Kenya,Machakos,Athi river,No,Nan,NaN,NaN,10-09-2020,10-09-2020,Negative,11-09-2020
16345,NIL,40.0,M,Nil,Kenya,Laikipia,Laikipia east,No,Nan,NTRH,NaN,18-06-2020,NaN,Negative,20-06-2020
23661,KEM-21-03-93738,42.0,F,NaN,Kenya,Kiambu,Kahoho,No,Nan,NaN,NaN,03-03-2021,NaN,Positive,03-05-2021
23662,KEM-21-03-93999,55.0,F,NaN,Kenya,Kiambu,Githunguri,No,Nan,NaN,General weakness,03-02-2021,NaN,Positive,03-10-2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23808,KEM-21-03-94425,17.0,M,NaN,Kenya,Kiambu,Thika,No,Nan,NaN,"Cough, Runny nose",03-04-2021,NaN,Positive,03-11-2021
23809,KEM-21-03-94736,16.0,M,NaN,Kenya,Nairobi,Roysambu,No,Nan,NaN,NaN,03-12-2021,NaN,Positive,16-03-2021
23810,KEM-21-03-94766,22.0,M,NaN,Kenya,Kiambu,Gatundu north,No,Nan,NaN,"Headache,cough",03-10-2021,NaN,Positive,15-03-2021
23811,KEM-21-03-95380,60.0,F,NaN,Kenya,Kiambu,Kabete,No,Nan,NaN,NaN,18-03-2021,NaN,Positive,19-03-2021


In [99]:
df_dts_cln[df_dts_cln['S_NUM'].str.contains('COVC') == False]

,S_NUM,AGE_YRS,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF
9011,COVE20,39.0,M,SECURITY GUARD,Kenya,Machakos,Athi river,No,Nan,NaN,NaN,10-09-2020,10-09-2020,Negative,11-09-2020
9012,COVE21,50.0,M,SECURITY GUARD,Kenya,Machakos,Athi river,No,Nan,NaN,NaN,10-09-2020,10-09-2020,Negative,11-09-2020
16345,NIL,40.0,M,Nil,Kenya,Laikipia,Laikipia east,No,Nan,NTRH,NaN,18-06-2020,NaN,Negative,20-06-2020
23661,KEM-21-03-93738,42.0,F,NaN,Kenya,Kiambu,Kahoho,No,Nan,NaN,NaN,03-03-2021,NaN,Positive,03-05-2021
23662,KEM-21-03-93999,55.0,F,NaN,Kenya,Kiambu,Githunguri,No,Nan,NaN,General weakness,03-02-2021,NaN,Positive,03-10-2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23808,KEM-21-03-94425,17.0,M,NaN,Kenya,Kiambu,Thika,No,Nan,NaN,"Cough, Runny nose",03-04-2021,NaN,Positive,03-11-2021
23809,KEM-21-03-94736,16.0,M,NaN,Kenya,Nairobi,Roysambu,No,Nan,NaN,NaN,03-12-2021,NaN,Positive,16-03-2021
23810,KEM-21-03-94766,22.0,M,NaN,Kenya,Kiambu,Gatundu north,No,Nan,NaN,"Headache,cough",03-10-2021,NaN,Positive,15-03-2021
23811,KEM-21-03-95380,60.0,F,NaN,Kenya,Kiambu,Kabete,No,Nan,NaN,NaN,18-03-2021,NaN,Positive,19-03-2021


11-09-2020

COVC9774
COVC9775
COVE20
COVE21
COVC9776
COVC9777


### **ID Misssing Data**

In [100]:
start = 0
df_missing = pd.DataFrame(columns=cols_rename)
for num in df_dts_cln['S_NUM'].sort_values():
#     curr_row = df_dts_cln[df_dts_cln['S_NUM']]
    try:
        curr = int(num.lstrip('COVC').lstrip('0'))
        
        if curr - start > 1:
            start_id = f'COVC{start}'
            cov_id = f'COVC{curr}'
            df_missing = df_missing.append(df_dts_cln[df_dts_cln['S_NUM'] == cov_names(start_id)])
            df_missing = df_missing.append(df_dts_cln[df_dts_cln['S_NUM'] == cov_names(cov_id)])
        start = curr
    except ValueError:
#         print(f'ValueError: {num}')
        pass

In [101]:
df_missing.head()

,S_NUM,AGE,AGE_UNIT,GEND,OCCU,NAT,COUNT_RES,S_COUNT_RES,TRAV_HIST,TRAV_FROM,QUAR_PLACE,SYMPS,DT_SAM_COLL,DT_SAM_RECEP,RESULT,DT_CONF,AGE_YRS
9476,COVC00835,NaN,NaN,M,Nil,Kenya,Nakuru,Bahati,No,Nan,Kiti Dispensary,Asymtomatic,09-06-2020,11-06-2020,Negative,12-06-2020,34.0
10804,COVC00835,NaN,NaN,M,Nil,Kenya,Nakuru,Bahati,No,Nan,Kiti Dispensary,Asymtomatic,09-06-2020,11-06-2020,Negative,12-06-2020,34.0
11651,COVC00930,NaN,NaN,M,Food handler,Kenya,Nairobi,Embakasi east,No,Nan,Nil,Asymtomatic,08-06-2020,14-06-2020,Negative,14-06-2020,27.0
12435,COVC01117,NaN,NaN,M,Nil,Kenya,Nairobi,Embakasi,No,Nan,Naivas Capital Centre,Asymtomatic,08-06-2020,15-06-2020,Negative,15-06-2020,30.0
13233,COVC01120,NaN,NaN,F,NaN,Kenya,Laikipia,Laikipia east,No,Nan,NTRH,NaN,14-06-2020,NaN,Negative,16-06-2020,32.0


In [102]:
(df_missing[['S_NUM', 'DT_SAM_COLL', 'DT_SAM_RECEP', 'DT_CONF']]
 .sort_values('S_NUM')
 .drop_duplicates('S_NUM', keep='last')
 .to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged-missingII.xlsx', index=False))

### **ID Missing Results**

In [103]:
(df_dts_cln[df_dts_cln['RESULT'].isin(['Positive', 'Negative']) == 
            False][['S_NUM', 'DT_SAM_COLL', 'DT_SAM_RECEP', 'DT_CONF', 'RESULT']]
.to_excel('/home/douso/Documents/TrendData/Results/ResultsMerged/COVID19-results-merged-results-missingII.xlsx', index=False))

### **Filter Samples Seq'd in seq10**

In [104]:
# df_waiting = pd.read_excel('/home/douso/Documents/RunSheets/BioRepository/cov-positiveCtsLocsIII-flt.xlsx')
# df_ext_seq10 = pd.read_excel('/home/douso/Documents/RunSheets/BioRepository/run10_extractionplates.xlsx')

In [105]:
# df_waiting.head()

In [106]:
# df_ext_seq10.head()

In [107]:
# ext_list = list(df_ext_seq10['Sample_Name'])

In [108]:
# df_waiting11 = df_waiting[df_waiting['Sample_Name'].isin(ext_list) == False]

In [109]:
# df_waiting11.to_excel('/home/douso/Documents/RunSheets/BioRepository/cov-positiveCtsLocsIII-flt2.xlsx', index=False)

In [110]:
# plt.style.available